In [1]:
import tensorflow as tf
import numpy as np
import json
import os
import csv
from dotenv import load_dotenv
load_dotenv()

2025-08-11 23:22:33.791130: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 23:22:34.199194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754929335.195419    1048 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754929354.553449    1048 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754929354.866157    1048 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

True

In [2]:
ML_Summer_School_ID = os.getenv('ML_Summer_School_ID')
print("Your Sudent ID is: " + ML_Summer_School_ID)


Your Sudent ID is: ML051_Kaung_Khant_Paing


In [3]:
BATCH_SIZE  = 32
input_img_size = (128,128)
input_shape = (128,128,3)


test_dir = r'/home/sumo/Proj/Practical-ML-by-WAI/6_deep_learning/CNN/cats_and_dogs_filtered/test'
model_path = f"./binary_classification/{ML_Summer_School_ID}_model.h5"
class_index_path = f'./binary_classification/{ML_Summer_School_ID}_class_indices.json'
mark_path = "./binary_classification/marks.txt"


In [4]:
model = tf.keras.models.load_model(model_path)
print(model.summary())

I0000 00:00:1754929360.486541    1048 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3584 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 128, 128, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │        32,769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,051 (148.64 KB)

 Trainable params: 37,953 (148.25 KB)

 Non-trainable params: 96 (384.00 B)

 Optimizer params: 2 (12.00 B)

None


In [5]:
# Load class indices from the JSON file
with open(class_index_path, 'r') as f:
    class_indices = json.load(f)
print("Class indices loaded:", class_indices)


Class indices loaded: {'cats': 0, 'dogs': 1}


In [6]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=input_img_size,
    batch_size=BATCH_SIZE,
    shuffle=False,  # Important: preserve label order for evaluation
    label_mode='int'  # returns integer labels
)


Found 248 files belonging to 2 classes.


In [7]:
# === Predict ===
y_true = []
y_pred = []


for images, labels in test_ds:
    predictions = model.predict(images)

    # If binary classification
    if predictions.shape[1] == 1:
        preds = (predictions > 0.5).astype(int).flatten()
    else:  # multi-class
        preds = np.argmax(predictions, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(preds)

# === Evaluate Accuracy ===
y_true = np.array(y_true)
y_pred = np.array(y_pred)

I0000 00:00:1754929362.043334    1279 service.cc:152] XLA service 0x795a6c0052f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754929362.043912    1279 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-08-11 23:22:42.077437: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1754929362.143399    1279 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


I0000 00:00:1754929364.189290    1279 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step


2025-08-11 23:22:45.510454: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
accuracy = np.mean(y_true == y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 60.89%


In [ ]:
with open(mark_path, "w") as f:
    f.write(str(int(accuracy * 100)))